In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(wiki_url)
# print(result)
soup = BeautifulSoup(result.text, 'html.parser')

content = soup.find('table', {'class': 'wikitable sortable'})
header_cols = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=header_cols)

for tr in content.find_all('tr'):
    r = []
    for td in tr.find_all('td'):
        r.append(td.text.strip())
    if len(r) == 3:
        df.loc[len(df)] = r
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df_clean = df[df['Borough']!='Not assigned']
df_clean.reset_index(inplace=True)
df_clean.drop(['index'], axis=1, inplace=True)
df_clean.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
geo_data_url = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(geo_data_url)
df_geo.rename(columns={'Postal Code': 'Postalcode'}, inplace=True)
df_geo.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_merged = pd.merge(df_geo, df_clean, on='Postalcode')
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto')]
print(df_toronto.shape)
df_toronto.head()

(39, 5)


,Postalcode,Latitude,Longitude,Borough,Neighborhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [7]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500, category_id=''):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        if category_id != '':
            url = url + '&categoryId={}'.format(category_id)
        
        # return only relevant information for each nearby venue
        for v in results:
            success = False
            try:
                category = v['categories'][0]['name']
                success = True
            except:
                pass

            if success:
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['name'], 
                    v['location']['lat'], 
                    v['location']['lng'],
                    v['categories'][0]['name']
                )])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
CLIENT_ID = 'ZHBKKVZ0YYSIUMWMXUZQDYDKRON0VNYGVQZ0WI4BPHKD5HHB' # your Foursquare ID
CLIENT_SECRET = 'MVZXK3XRCCSACUFMX5O2A2M240D0P3GFHL0A13X5SVEEBULI' # your Foursquare Secret
VERSION = '20180604'

japanese_restaurant_venues = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'], category_id='4bf58dd8d48988d111941735')

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [9]:
japanese_restaurant_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Kew-Balmy Beach,43.667372,-79.295312,Beach
2,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Ivan Forrest Gardens,43.671669,-79.292573,Park


In [10]:
japanese_restaurant_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,84,84,84,84,84,84
"Brockton, Parkdale Village, Exhibition Place",92,92,92,92,92,92
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",79,79,79,79,79,79
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",62,62,62,62,62,62
Central Bay Street,94,94,94,94,94,94
Christie,94,94,94,94,94,94
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",93,93,93,93,93,93
Davisville,79,79,79,79,79,79


In [11]:
toronto_onehot = pd.get_dummies(japanese_restaurant_venues[['Venue Category']], prefix="", prefix_sep="")
# toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=japanese_restaurant_venues['Neighborhood'] )

print(toronto_onehot.shape)
toronto_onehot.head()

(3284, 410)


,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Campaign Office,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Theater,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Corporate Cafeteria,Corporate Coffee Shop,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Music Sch

In [12]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Campaign Office,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Theater,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Corporate Cafeteria,Corporate Coffee Shop,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Music Sch

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Office,Building,Parking,Tech Startup,Residential Building (Apartment / Condo),Breakfast Spot,Korean Restaurant,Movie Theater,Assisted Living,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Office,Residential Building (Apartment / Condo),Tech Startup,Conference Room,Building,Advertising Agency,Café,Bar,Coworking Space,Convenience Store
2,"Business reply mail Processing Centre, South C...",Building,Office,Light Rail Station,Convenience Store,Medical Center,Butcher,Fast Food Restaurant,Theater,Restaurant,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Gate,Airport Service,Moving Target,Airport Terminal,Park,Airport Lounge,Harbor / Marina,Airport,Boat or Ferry,General Travel
4,Central Bay Street,Hospital,Hospital Ward,Medical Center,Office,Emergency Room,Coffee Shop,Pharmacy,Deli / Bodega,Food Court,Fast Food Restaurant


In [15]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 2, 4, 1, 3, 0, 1, 3], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,1,School,Park,Building,Playground,Laundry Service,Residential Building (Apartment / Condo),Jewelry Store,Salon / Barbershop,Doctor's Office,Martial Arts School
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",1,Greek Restaurant,Spa,Salon / Barbershop,Miscellaneous Shop,Gym / Fitness Center,Women's Store,Office,Health Food Store,Pharmacy,Fruit & Vegetable Store
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West",1,Convenience Store,Park,Office,Laundry Service,Pet Store,Rental Car Location,Church,Car Wash,Light Rail Station,Pizza Place
43,M4M,43.659526,-79.340923,East Toronto,Studio District,1,Building,Office,Automotive Shop,Coffee Shop,Restaurant,Nail Salon,Spa,Seafood Restaurant,Furniture / Home Store,Moving Target
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,1,College Classroom,College Auditorium,School,Housing Development,Fast Food Restaurant,Building,Bus Line,Park,College Theater,Parking


In [17]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.6534817, -79.3839347.


In [18]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [19]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,43.654260,0,Office,Automotive Shop,Furniture / Home Store,Building,Coffee Shop,Art Gallery,Food Truck,Italian Restaurant,Light Rail Station,Auto Dealership
55,43.651494,0,Office,Residential Building (Apartment / Condo),Event Space,Building,Coffee Shop,Spa,Church,Nightclub,Clothing Store,Rental Car Location
56,43.644771,0,Office,Building,Parking,Tech Startup,Residential Building (Apartment / Condo),Breakfast Spot,Korean Restaurant,Movie Theater,Assisted Living,Hotel
60,43.647177,0,Office,Coffee Shop,Building,Café,Park,Restaurant,Pharmacy,Italian Restaurant,Coworking Space,Event Space
61,43.648198,0,Office,Coffee Shop,Financial or Legal Service,Salon / Barbershop,Food Court,Bagel Shop,Restaurant,Bus Stop,Building,Dentist's Office
69,43.646435,0,Office,Building,Tech Startup,Residential Building (Apartment / Condo),Gym,Bar,Pub,Hotel,Gym / Fitness Center,Café
78,43.636847,0,Office,Residential Building (Apartment / Condo),Tech Startup,Conference Room,Building,Advertising Agency,Café,Bar,Coworking Space,Convenience Store


In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,43.676357,1,School,Park,Building,Playground,Laundry Service,Residential Building (Apartment / Condo),Jewelry Store,Salon / Barbershop,Doctor's Office,Martial Arts School
41,43.679557,1,Greek Restaurant,Spa,Salon / Barbershop,Miscellaneous Shop,Gym / Fitness Center,Women's Store,Office,Health Food Store,Pharmacy,Fruit & Vegetable Store
42,43.668999,1,Convenience Store,Park,Office,Laundry Service,Pet Store,Rental Car Location,Church,Car Wash,Light Rail Station,Pizza Place
43,43.659526,1,Building,Office,Automotive Shop,Coffee Shop,Restaurant,Nail Salon,Spa,Seafood Restaurant,Furniture / Home Store,Moving Target
44,43.728020,1,College Classroom,College Auditorium,School,Housing Development,Fast Food Restaurant,Building,Bus Line,Park,College Theater,Parking
46,43.715383,1,Shoe Store,Clothing Store,Men's Store,Furniture / Home Store,Health & Beauty Service,Cosmetics Shop,Marijuana Dispensary,Metro Station,Accessories Store,General Entertainment
47,43.704324,1,Café,Coffee Shop,Dessert Shop,Italian Restaurant,French Restaurant,Bookstore,Spa,Salon / Barbershop,Antique Shop,Laundry Service
48,43.689574,1,Office,Building,Residential Building (Apartment / Condo),Park,Bridge,Gym,Bank,Tennis Court,Trail,Gym / Fitness Center
51,43.667967,1,Café,Flower Shop,Coffee Shop,Office,Pharmacy,Pizza Place,Restaurant,Indian Restaurant,Bakery,Tech Startup
54,43.657162,1,College Lab,College Administrative Building,University,College Arts Building,General College & University,Parking,College Academic Building,Coffee Shop,Student Center,College Communications Building


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,43.628947,2,Airport Gate,Airport Service,Moving Target,Airport Terminal,Park,Airport Lounge,Harbor / Marina,Airport,Boat or Ferry,General Travel


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,43.712751,3,Residential Building (Apartment / Condo),Office,Gym,Strip Club,Dog Run,Medical Center,Building,Italian Restaurant,Tea Room,Flower Shop
49,43.686412,3,Residential Building (Apartment / Condo),Office,Building,Doctor's Office,Dentist's Office,Medical Center,Light Rail Station,Government Building,Embassy / Consulate,Café
50,43.679563,3,Residential Building (Apartment / Condo),Office,Park,Trail,Building,Bridge,Music Venue,Dog Run,Salon / Barbershop,Conference Room
52,43.665860,3,Residential Building (Apartment / Condo),Office,Spa,Building,Gym,Doctor's Office,Bubble Tea Shop,Hotel,Thrift / Vintage Store,Pub
59,43.640816,3,Residential Building (Apartment / Condo),Office,Coffee Shop,Building,Light Rail Station,Parking,Doctor's Office,Fried Chicken Joint,Tea Room,Bank
65,43.672710,3,Residential Building (Apartment / Condo),Office,Bed & Breakfast,Building,General Entertainment,Gym,Metro Station,Medical Center,Café,Hotel


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,43.657952,4,Hospital,Hospital Ward,Medical Center,Office,Emergency Room,Coffee Shop,Pharmacy,Deli / Bodega,Food Court,Fast Food Restaurant
